# Presidio PII Pseudonymizer dla języka polskiego

Narzędzie do wykrywania i pseudonimizacji danych osobowych (PII) w tekstach polskojęzycznych. Wykorzystuje [Microsoft Presidio](https://microsoft.github.io/presidio/) do wykrywania PII oraz [Faker](https://faker.readthedocs.io/) do generowania realistycznych, deterministycznych zamienników.

## 🔍 Jak to działa

Rozwiązanie składa się z trzech głównych komponentów:

### 1. Wykrywanie PII (Presidio Analyzer)

```
Tekst wejściowy → [NLP Engine (spaCy)] → [Rozpoznawacze] → Lista encji PII
```

**Silnik NLP (spaCy)** przetwarza tekst i wykonuje:
- Tokenizację (podział na słowa)
- Lematyzację (sprowadzenie do formy podstawowej)
- **Named Entity Recognition (NER)** - rozpoznawanie nazwanych encji (osoby, organizacje, miejsca)

**Rozpoznawacze (Recognizers)** to moduły wykrywające konkretne typy PII:
- **SpacyRecognizer** - wykorzystuje etykiety NER z modelu spaCy (np. `persName` → `PERSON`)
- **Rozpoznawacze regex** - wykrywają wzorce jak PESEL, NIP, email na podstawie wyrażeń regularnych
- **Słowa kontekstowe** - zwiększają pewność wykrycia gdy w pobliżu znajdują się słowa jak "pesel", "telefon", "email"

### 2. Generowanie zamienników (Faker)

```
Encja PII → [Hash(salt + wartość)] → [Seed] → [Faker] → Pseudonim
```

Mechanizm zapewnia **determinizm** - ta sama wartość wejściowa zawsze daje ten sam pseudonim:

1. Obliczamy hash SHA-256 z kombinacji: `salt + oryginalna_wartość + typ_encji`
2. Fragment hasha używamy jako seed dla generatora Faker
3. Faker generuje realistyczną, fikcyjną wartość (imię, nazwę firmy, email itp.)

Dla numerów PESEL, NIP, REGON stosujemy własne generatory z **poprawnymi sumami kontrolnymi**.

### 3. Zastępowanie w tekście

```
Tekst + Lista encji → [Sortowanie malejąco po pozycji] → [Zamiana od końca] → Tekst zanonimizowany
```

Zamiany wykonujemy **od końca tekstu**, aby nie popsuć indeksów pozycji wcześniejszych encji.

### Schemat przepływu danych

```
┌─────────────────────────────────────────────────────────────────────┐
│                         TEKST WEJŚCIOWY                             │
│  "Jan Kowalski, PESEL: 90010112345, email: jan@example.com"         │
└─────────────────────────────────────────────────────────────────────┘
                                   │
                                   ▼
┌─────────────────────────────────────────────────────────────────────┐
│                      1. ANALIZA (Presidio)                          │
│  ┌─────────────┐    ┌──────────────────┐    ┌──────────────────┐   │
│  │ spaCy NLP   │───▶│ SpacyRecognizer  │───▶│ PERSON: 0-12     │   │
│  │ (pl_core_   │    │ (NER: persName)  │    │ "Jan Kowalski"   │   │
│  │  news_lg)   │    └──────────────────┘    └──────────────────┘   │
│  └─────────────┘    ┌──────────────────┐    ┌──────────────────┐   │
│                     │ PeselRecognizer  │───▶│ PL_PESEL: 21-32  │   │
│                     │ (regex + kontekst│    │ "90010112345"    │   │
│                     └──────────────────┘    └──────────────────┘   │
│                     ┌──────────────────┐    ┌──────────────────┐   │
│                     │ EmailRecognizer  │───▶│ EMAIL: 41-57     │   │
│                     │ (regex)          │    │ "jan@example.com"│   │
│                     └──────────────────┘    └──────────────────┘   │
└─────────────────────────────────────────────────────────────────────┘
                                   │
                                   ▼
┌─────────────────────────────────────────────────────────────────────┐
│                   2. GENEROWANIE ZAMIENNIKÓW                        │
│                                                                     │
│  "Jan Kowalski" ──hash──▶ seed:1234 ──Faker──▶ "Tadeusz Elwart"    │
│  "90010112345"  ──hash──▶ seed:5678 ──PESEL──▶ "97050447064"       │
│  "jan@example.com" ─hash─▶ seed:9012 ─Faker─▶ "dorobekemil@..."    │
└─────────────────────────────────────────────────────────────────────┘
                                   │
                                   ▼
┌─────────────────────────────────────────────────────────────────────┐
│                    3. ZASTĘPOWANIE W TEKŚCIE                        │
│  (od końca, aby zachować indeksy)                                   │
│                                                                     │
│  "Tadeusz Elwart, PESEL: 97050447064, email: dorobekemil@..."      │
└─────────────────────────────────────────────────────────────────────┘
```

## 🧠 Modele spaCy

spaCy oferuje trzy modele dla języka polskiego, różniące się rozmiarem i dokładnością:

| Model | Rozmiar | Wektory | Dokładność NER | Użycie |
|-------|---------|---------|----------------|--------|
| `pl_core_news_sm` | ~15 MB | ❌ Brak | ~85% F1 | Szybkie prototypowanie, ograniczone zasoby |
| `pl_core_news_md` | ~50 MB | ✅ 20k kluczy | ~87% F1 | Balans między rozmiarem a dokładnością |
| `pl_core_news_lg` | ~550 MB | ✅ 500k kluczy | ~89% F1 | **Zalecany** - najlepsza jakość |

### Porównanie modeli

```
┌─────────────────────────────────────────────────────────────────┐
│                    DOKŁADNOŚĆ vs ROZMIAR                        │
│                                                                 │
│  Dokładność │                                    ● pl_core_lg   │
│     NER     │                        ● pl_core_md               │
│             │            ● pl_core_sm                           │
│             │                                                   │
│             └───────────────────────────────────────────────────│
│                         Rozmiar modelu / Czas ładowania         │
└─────────────────────────────────────────────────────────────────┘
```

### Etykiety NER w polskich modelach

Polski model spaCy używa **specyficznych etykiet** różnych od standardowych:

| Etykieta spaCy (PL) | Opis | Mapowanie Presidio |
|---------------------|------|-------------------|
| `persName` | Imię i nazwisko osoby | `PERSON` |
| `orgName` | Nazwa organizacji/firmy | `ORGANIZATION` |
| `placeName` | Nazwa miejsca | `LOCATION` |
| `geogName` | Nazwa geograficzna | `LOCATION` |
| `date` | Data | `DATE_TIME` |
| `time` | Czas | `DATE_TIME` |

> ⚠️ **Ważne**: Standardowe modele angielskie używają etykiet jak `PERSON`, `ORG`, `GPE`. 
> Polski model używa `persName`, `orgName`, `placeName`. 
> Mapowanie jest skonfigurowane w pliku `languages-config.yml`.


### Zmiana modelu

Aby użyć innego modelu, edytuj `languages-config.yml`:

```yaml
models:
  - lang_code: pl
    model_name: pl_core_news_md  # zmień na sm, md lub lg
```
### Wybór modelu - rekomendacje

| Scenariusz | Zalecany model |
|------------|----------------|
| Produkcja - wysoka jakość | `pl_core_news_lg` |
| Środowisko z ograniczoną pamięcią | `pl_core_news_md` |
| Szybkie testy / CI/CD | `pl_core_news_sm` |
| Przetwarzanie dużych wolumenów | `pl_core_news_md` (kompromis) |

## 📊 Obsługiwane typy PII

### Encje wykrywane przez NLP (spaCy)

| Typ | Opis | Przykład |
|-----|------|----------|
| `PERSON` | Imię i nazwisko | Jan Kowalski |
| `ORGANIZATION` | Nazwa firmy/organizacji | Drutex Sp.z.o.o. |
| `LOCATION` | Lokalizacja | Warszawa, Polska |
| `DATE_TIME` | Data i czas | 15 marca 2024 |

### Encje wykrywane przez regex (polskie formaty)

| Typ | Opis | Przykład | Walidacja |
|-----|------|----------|-----------|
| `PL_PESEL` | Numer PESEL | 90010112345 | ✅ Suma kontrolna |
| `PL_NIP` | Numer NIP | 123-456-78-90 | ✅ Suma kontrolna |
| `PL_REGON` | Numer REGON | 123456789 | ✅ Suma kontrolna |
| `PL_ID_CARD` | Dowód osobisty | ABC123456 | ❌ |
| `PL_PASSPORT` | Paszport | AB1234567 | ❌ |
| `PL_POSTAL_CODE` | Kod pocztowy | 00-001 | ❌ |
| `PL_PHONE` | Telefon komórkowy | +48 123 456 789 | ❌ |
| `PL_BANK_ACCOUNT` | Numer konta | 12 3456 7890... | ❌ |

### Encje predefiniowane Presidio

| Typ | Opis | Przykład |
|-----|------|----------|
| `EMAIL_ADDRESS` | Adres email | jan@example.com |
| `PHONE_NUMBER` | Numer telefonu | +48123456789 |
| `CREDIT_CARD` | Karta kredytowa | 4111111111111111 |
| `IBAN_CODE` | Numer IBAN | PL12345678901234567890123456 |
| `IP_ADDRESS` | Adres IP | 192.168.1.1 |
| `URL` | Adres URL | https://example.com |

In [4]:
from pseudonymizer import Pseudonymizer

# Utwórz instancję
ps = Pseudonymizer(salt="moj-sekretny-klucz")

# Pseudonimizacja tekstu
text = "Jan Kowalski, numer PESEL: 90010112345, email: jan@example.com"
result = ps.pseudonymize(text)
print(result)


Liwia Machoń, numer PESEL: 88112010386, email: adrian46@example.org


In [5]:
result = ps.pseudonymize_with_details(text)

print(f"Oryginał: {result.original_text}")
print(f"Pseudonim: {result.pseudonymized_text}")
print(f"Znaleziono encji: {result.entities_found}")

for repl in result.replacements:
    print(f"  {repl.entity_type}: '{repl.original}' → '{repl.replacement}'")

Oryginał: Jan Kowalski, numer PESEL: 90010112345, email: jan@example.com
Pseudonim: Liwia Machoń, numer PESEL: 88112010386, email: adrian46@example.org
Znaleziono encji: 3
  EMAIL_ADDRESS: 'jan@example.com' → 'adrian46@example.org'
  PL_PESEL: '90010112345' → '88112010386'
  PERSON: 'Jan Kowalski' → 'Liwia Machoń'


# Praktyczne wykorzystanie Presidio do pseudonimizacji danych osobowych

Oto przykładowa, krótka testowa konwersacja w banku

***

**Pracownik banku:** Dzień dobry, w czym mogę pomóc?  
**Klient:** Dzień dobry, chciałbym sprawdzić stan mojego konta.  
**Pracownik banku:** Oczywiście. Proszę podać numer konta.  
**Klient:** Numer konta to 12 3456 7890 1234 5678 9012 3456.  
**Pracownik banku:** Dziękuję. Proszę jeszcze o PESEL i adres zamieszkania do weryfikacji.  
**Klient:** PESEL: 85010112345, adres: ul. Zielona 15, 00-123 Warszawa.  
**Pracownik banku:** Wszystko się zgadza. Stan konta wynosi 12 450,75 zł. Czy mogę pomóc w czymś jeszcze?  
**Klient:** Nie, dziękuję. Miłego dnia!  
**Pracownik banku:** Dziękuję, również życzę miłego dnia.



In [7]:
input_text = """**Pracownik banku:** Dzień dobry, w czym mogę pomóc?  
**Klient:** Dzień dobry, chciałbym sprawdzić stan mojego konta.  
**Pracownik banku:** Oczywiście. Proszę podać numer konta.  
**Klient:** Numer konta to 12 3456 7890 1234 5678 9012 3456.  
**Pracownik banku:** Dziękuję. Proszę jeszcze o PESEL i adres zamieszkania do weryfikacji.  
**Klient:** PESEL: 85010112345, adres: ul. Zielona 15, 00-123 Warszawa.  
**Pracownik banku:** Wszystko się zgadza. Stan konta wynosi 12 450,75 zł. Czy mogę pomóc w czymś jeszcze?  
**Klient:** Nie, dziękuję. Miłego dnia!  
**Pracownik banku:** Dziękuję, również życzę miłego dnia."""
result = ps.pseudonymize(input_text)
print(result)

**Pracownik banku:** Dzień dobry, w czym mogę pomóc?  
**Klient:** Dzień dobry, chciałbym sprawdzić stan mojego konta.  
**Pracownik banku:** Oczywiście. Proszę podać numer konta.  
**Klient:** Numer konta to 13 0842 4539 7131 0883 4664 9863.  
**Pracownik banku:** Dziękuję. Proszę jeszcze o PESEL i adres zamieszkania do weryfikacji.  
**Klient:** PESEL: 81040889651, adres: Pruszcz Gdański Szczytno 15, 70-144 Środa Wielkopolska.  
**Pracownik banku:** Wszystko się zgadza. Stan konta wynosi 12 450,75 zł. Czy mogę pomóc w czymś jeszcze?  
**Klient:** Nie, dziękuję. Miłego dnia!  
**Pracownik banku:** Dziękuję, również życzę miłego dnia.
